# 독성 예측

## deepchem 설치

In [ ]:
!pip install deepchem

     |████████████████████████████████| 608 kB 5.4 MB/s 
     |████████████████████████████████| 22.5 MB 59.4 MB/s 


In [ ]:
import numpy as np
import deepchem as dc

# Load the data.

tox21_tasks, tox21_datasets, transformers = dc.molnet.load_tox21()
train_dataset, valid_dataset, test_dataset = tox21_datasets

In [ ]:
# 독성 반응과 관계된 단백질(target) 12종에 대한 실험 데이터를 제공한다
# 실험 데이터: 각 분자와 해당 단백질의 결합력
tox21_tasks

['NR-AR',
 'NR-AR-LBD',
 'NR-AhR',
 'NR-Aromatase',
 'NR-ER',
 'NR-ER-LBD',
 'NR-PPAR-gamma',
 'SR-ARE',
 'SR-ATAD5',
 'SR-HSE',
 'SR-MMP',
 'SR-p53']

In [ ]:
# train_dataset 등은 DiskDataset 객체로 만들어져 있다
# X, y, w 값이 들어 있다
# 데이터를 반복해서 불러올 필요가 없다
print(train_dataset.X.shape)
print(train_dataset.y.shape)

(6264, 1024)
(6264, 12)


In [ ]:
# 결측치 처리
# 누락된 데이터가 반영되지 않도록 가중치 벡터를 0으로 처리한는 방법을 사용한다 (결측치를 나타낸다)

In [ ]:
# 총 6264*12 개의 가중치 파라미터 중에 값이 0인 것의 개수를 세어본다
np.count_nonzero(train_dataset.w == 0)

11521

In [ ]:
# 원시 데이터를 수정한 객체
# 아래 내용을 보면 불균형 데이터를 보완한 것을 알 수 있다
# 가중치 행렬을 조정하는 방법을 사용한다 (클래스별 총 가중치가 동일하도록 조정)
transformers

In [ ]:
# MLP 분류 모델 사용해보겠다
# 아래는 레이어 하나, 크기 1000인 MLP 모델을 정의했다

model = dc.models.MultitaskClassifier(n_tasks=12, n_features=1024, layer_sizes=[1000])
model.fit(train_dataset, nb_epoch=10)

# 12개 작업의 AUC를 구하고 평균을 사용한다

metric = dc.metrics.Metric(dc.metrics.roc_auc_score, np.mean)
train_scores = model.evaluate(train_dataset, [metric], transformers)
test_scores = model.evaluate(test_dataset, [metric], transformers)
print(train_scores)
print(test_scores)

{'mean-roc_auc_score': 0.9586985949150856}
{'mean-roc_auc_score': 0.6798976981349182}
